# Dependencies

In [1]:
!pip3 install pyzmq cryptography sympy pycryptodome

Defaulting to user installation because normal site-packages is not writeable


# Initialization

In [97]:
import sympy
import random
import zmq
import threading
from Crypto.Cipher import AES
from Crypto.Util import Counter
from Crypto.Random import get_random_bytes
import base64

In [2]:
# math
BITS_OF_PRIME_GROUP = 512

# socket
LOCAL_PORT = 4080
SERVER_HOST = "localhost"
SERVER_PORT = 4080

# Basics

## Primes and Modular arithmetics

In [225]:
def get_random_prime():
    """
    Generate a random prime number of the specified number of bits.
    """

    lower_bound = 2**(BITS_OF_PRIME_GROUP - 1)
    upper_bound = 2**BITS_OF_PRIME_GROUP - 1

    prime = sympy.randprime(lower_bound, upper_bound)
    
    return prime

def find_generator(prime):
    """
    For every prime divisor q of p -1 check if g^{(p-1)/q} ≡ 1 mod p. If that happens, discard g
    If it survives till the last prime divisor of p - 1 is a generator.
    """

    while 1:
        candidate = pow(random.randint(2, prime - 1), 2, prime)

        # avoid g=2 because of Bleichenbacher's attack described # in "Generating
        # ElGamal signatures without knowning the secret key", 1996
        if candidate in (1, 2):
            continue

        # Discard g if it divides p-1 because of the attack described
        # in Note 11.67 (iii) in HAC
        if (prime - 1) % candidate == 0:
            continue

        # g^{-1} must not divide p-1 because of Khadir's attack described in
        # "Conditions of the generator for forging ElGamal signature", 2011
        ginv = inverse(candidate, prime)
        if (prime - 1) % ginv == 0:
            continue

        # found
        break

    return candidate

def inverse(num, prime):
    """
    Multiplicative inverse.
    """

    return pow(num, prime - 2, prime)

def bitwise_xor(x, y):
    """
    Perform bitwise XOR given two byte sequences.
    """

    x_int = int.from_bytes(x, byteorder="big")
    y_int = int.from_bytes(y, byteorder="big")

    return (x_int ^ y_int).to_bytes(len(x), byteorder="big")

## Secret Key Encryption

The cryptography library contains Fernet, an implementation of symmetric (secret key) authenticated cryptography.

In [226]:
###############################################
# IMPLEMENTATION OF AES KEY ENCRYPTION SCHEME #
###############################################

def aes_generate_key():
    """
    Generate a 256-bit (32-byte) secret key for AES encryption.
    https://github.com/Legrandin/pycryptodome/blob/master/lib/Crypto/Cipher/AES.py
    https://pycryptodome.readthedocs.io/en/latest/src/cipher/aes.html
    """

    # AES-256 requires a 32-byte key
    key = get_random_bytes(32)

    return key

def pad(msg):
    """
    Pad message to be a multiple of AES block size (16 bytes).
    """

    # get how much padding
    padding_length = AES.block_size - len(msg) % AES.block_size

    # apply padding
    return msg + bytes([padding_length] * padding_length)

def unpad(msg):
    """
    Remove padding from decrypted message.
    """

    return msg[:-msg[-1]]

def aes_encrypt(msg, key):
    """
    Encrypts a message with a key using AES-256 in CTR mode.
    """

    # pad message
    msg = pad(msg)

    # init AES
    cipher = AES.new(key, AES.MODE_CTR)

    # encrypt message
    encrypted_msg = cipher.encrypt(msg)

    # return concatenation with nonce
    return cipher.nonce + encrypted_msg
    

def aes_decrypt(token, key):
    """
    Decrypts a message with a key using AES-256 in CBC mode.
    """

    # extract nonce and encrypted message
    nonce = token[:AES.block_size // 2]
    encrypted_msg = token[AES.block_size // 2:]

    # init AES
    cipher = AES.new(key, AES.MODE_CTR, nonce=nonce)

    # decrypt
    decrypted_msg = cipher.decrypt(encrypted_msg)

    # unpad
    decrypted_msg = unpad(decrypted_msg)
    return decrypted_msg

In [227]:
#####################################
# TEST OF AES KEY ENCRYPTION SCHEME #
#####################################

# test message
input_msg = b"Hello, World!"

# generate random key
key = aes_generate_key()

# encrypt the test message
token = aes_encrypt(input_msg, key)

# decrypt the test message
output_msg = aes_decrypt(token, key)

# print results
print("Original message: ", input_msg.decode("utf-8"))
print("Encrypted message: ", base64.b64encode(token))
print("Decrypted message: ", output_msg.decode("utf-8"))

Original message:  Hello, World!
Encrypted message:  b'XZoeVc+kCAdqbf1HsDDAylpqPXJSBZK3'
Decrypted message:  Hello, World!


## Verifiable Encryption

In [228]:
###########################################
# IMPLEMENTATION OF VERIFIABLE ENCRYPTION #
###########################################

def aes_as_doubling_prf(msg, key):
    """
    USe AES as a pseudo-random function.
    """

    counter1 = Counter.new(112, prefix=b'\x00\x01', initial_value=1)
    counter2 = Counter.new(112, prefix=b'\x00\x02', initial_value=2)

    # init AES
    cipher1 = AES.new(key, AES.MODE_CTR, counter=counter1)
    cipher2 = AES.new(key, AES.MODE_CTR, counter=counter2)

    # encrypt message
    encrypted_msg1 = cipher1.encrypt(msg)
    encrypted_msg2 = cipher2.encrypt(msg)

    return encrypted_msg1 + encrypted_msg2

def verifiable_encrypt(x, key):
    """
    Perform encryption verifiable by adding padding with zeros in such a way that when
    decrypting you can understand if the decryption has been successful or not.

    Nomenclature is the same used in the slides.
    """

    # create random noise
    r = get_random_bytes(len(x))

    # add padding
    x += b'\x00' * len(x)

    prf_output = aes_as_doubling_prf(r, key)

    s = bitwise_xor(prf_output, x)

    return r + s

def verifiable_decrypt(c, key):
    """
    Perform verifiable decryption
    """

    # unpack inputs
    r = c[:len(c) // 3]
    s = c[len(c) // 3:]

    # recompute prf
    prf_output = aes_as_doubling_prf(r, key)

    # decrypt function
    x = bitwise_xor(prf_output, s)

    n = len(x) // 2

    # check if decryption is correct

    if x[n:] == b'\x00' * n:
        return x[:n]

    else:
        return None

In [229]:
#################################
# TEST OF VERIFIABLE ENCRYPTION #
#################################

msg = get_random_bytes(16)

key1 = get_random_bytes(32)
key2 = get_random_bytes(32)

ciphertext = verifiable_encrypt(msg, key1)

decryption1 = verifiable_decrypt(ciphertext, key1)
decryption2 = verifiable_decrypt(ciphertext, key2)

print(f"Original message           : {msg}")
print(f"Decryption with correct key: {decryption1}")
print(f"Decryption with wrong key: {decryption2}")

Original message           : b'8\xf8h\x98\xd3W\xba\xb7\xe0\x08Kic\xa7Li'
Decryption with correct key: b'8\xf8h\x98\xd3W\xba\xb7\xe0\x08Kic\xa7Li'
Decryption with wrong key: None


## Double Verifiable Encryption

In [230]:
####################################################
# IMPLEMENTATION OF DOUBLE VERIFIABLE ENCRCRYPTION #
####################################################

def double_verifiable_encrypt(msg, key_x, key_y):
    """
    Function for verifiable double encryption that will be needed for creating the
    garbled tables.
    """

    # first perform encryption with y
    mid_ciphertext = verifiable_encrypt(msg, key_y)

    # then perform encryption with x
    return verifiable_encrypt(mid_ciphertext, key_x)

def double_verifiable_decrypt(ciphertext, key_x, key_y):
    """
    Function for verifiable double decryption that will be needed for computing the
    output of the garbled tables.
    """

    # first perform decryption with x
    mid_ciphertext = verifiable_decrypt(ciphertext, key_x)

    if mid_ciphertext is None:
        # if first decryption goes wrong return None
        return None
    
    else:
        # then perform decryption wih y
        return verifiable_decrypt(mid_ciphertext, key_y)

In [231]:
########################################
# TEST OF VERIFIABLE DOUBLE ENCRYPTION #
########################################

msg = get_random_bytes(16)

key1 = get_random_bytes(32)
key2 = get_random_bytes(32)

ciphertext = double_verifiable_encrypt(msg, key1, key2)

decryption = double_verifiable_decrypt(ciphertext, key1, key2)

print(f"Original message           : {msg}")
print(f"Decryption with correct key: {decryption}")

Original message           : b'\xc1\x8a2\x92\x9e\x82\x1d5\xc2@\xba\\+[b\xe0'
Decryption with correct key: b'\xc1\x8a2\x92\x9e\x82\x1d5\xc2@\xba\\+[b\xe0'


## Public Key Encryption

Implementation of ElGamal encryption scheme.

In [232]:
###############################################
# IMPLEMENTATION OF ELGAMAL ENCRYPTION SCHEME #
###############################################

def elgamal_prime_and_generator():
    """
    Generate random prime and genrator for ElGamal encryption scheme.
    The naming scheme follows what has been used in the course slides.
    """

    # generate random prime that defines the group
    p = get_random_prime()

    # find a generator for the group
    g = find_generator(p)

    return p, g

def elgamal_generate_keys(p_g = None):
    """
    Generate public and private keys for the ElGamal encryption scheme.
    The naming scheme follows what has been used in the course slides.
    """

    # get prime and generator
    if p_g is None:
        p, g = elgamal_prime_and_generator()

    else:
        p, g = p_g

    # generate a private key as a random number
    x = random.randint(1, p - 1)

    # calculate the public key as g^x mod p
    h = pow(g, x, p)

    return (g, p, h), x

def elgamal_encrypt(msg, public_key):
    """
    Encrypt a message with the ElGamal encryption scheme.
    """

    # from bytes to integer
    msg = int.from_bytes(msg, byteorder="big")

    # unpack public key
    g, p, h = public_key

    # generate a random number
    r = random.randint(1, p - 1)

    # calculate the two powers of the ciphertext
    gr = pow(g, r, p)
    hr = pow(h, r, p)

    # calculate the ciphertext
    c = (gr, (hr * msg) % p)

    return c

def elgamal_decrypt(c, private_key, public_key):
    """
    Decrypt a message with the ElGamal encryption scheme.
    """

    # unpack private key
    x = private_key

    # unpack public key
    g, p, h = public_key

    # unpack ciphertext
    c1, c2 = c

    # decrypt message
    m = (c2 * inverse(pow(c1, x, p), p)) % p

    # from integer to bytes
    m = m.to_bytes((m.bit_length() + 7) // 8, byteorder="big")

    return m

In [233]:
#####################################
# TEST OF ELGAMAL ENCRYPTION SCHEME #
#####################################

# test message
input_msg = b"Hello, World!"

# generate keys
public_key, private_key = elgamal_generate_keys()

# encrypt message
ciphertext = elgamal_encrypt(input_msg, public_key)

# decrypt message
output_msg = elgamal_decrypt(ciphertext, private_key, public_key)

# print results
print("Original message:", input_msg.decode())
print("Encrypted message:", ciphertext)
print("Decrypted message:", output_msg.decode())

Original message: Hello, World!
Encrypted message: (11077874484427170073264328600719297029931700924388270305865397677986970454803257096028784973419984407351119186603343998295472432607008710037523702578603490, 6371185120220155458260559776048367918929168797190435629826483334706677404840147615665333519114732924301265050833812458601561264184401180862335547840851739)
Decrypted message: Hello, World!


## Socket Communication

In [234]:
##########################################
# IMPLEMENTATION OF SOCKET COMMUNICATION #
##########################################

class Socket:
    def __init__(self, socket_type, address):
        """
        Initialize a ZeroMQ Socket.
        socket_type can be zmq.REQ, zmq.REP, zmq.PUB, zmq.SUB
        """

        # init socket settings
        self.context = zmq.Context.instance()
        self.socket = self.context.socket(socket_type)
        self.address = address

        if socket_type == zmq.REP:
            self.bind()
        elif socket_type == zmq.REQ:
            self.connect()
    
    def bind(self):
        """
        Bind the socket to given address.
        """

        self.socket.bind(self.address)
    
    def connect(self):
        """
        Connect the socket to a given address.
        """

        self.socket.connect(self.address)
    
    def send(self, msg):
        """
        Send a message through the socket.
        """

        self.socket.send_pyobj(msg)
    
    def receive(self):
        """
        Receive a message from the socket.
        """

        return self.socket.recv_pyobj()
    
    def close(self):
        """
        Close the socket.
        """

        self.socket.close()
    
    def __del__(self):
        """
        Destructor to close the socket.
        """

        self.close()

In [235]:
################################
# TEST OF SOCKET COMMUNICATION #
################################

# create a server and a client sockets
server = Socket(socket_type=zmq.REP, address=f"tcp://{SERVER_HOST}:{SERVER_PORT}")
client = Socket(socket_type=zmq.REQ, address=f"tcp://{SERVER_HOST}:{SERVER_PORT}")

# send a message from client to server
client.send("Hello, Server!")
print("Client received:", server.receive())

# send a message from server to client
server.send("Hello, Client!")
print("Server received:", client.receive())

# close the sockets
server.close()
client.close()

Client received: Hello, Server!
Server received: Hello, Client!


# Oblivious Transfer

## Passive Security

Implementation of a 1-out-of-2 Oblivious Transfer protocol with passive security.

In [236]:
class ObliviousTransfer_PS:
    """
    Implement the oblivious transfer protocol with passive security.

    In passive security, Bob sends both public keys to Alice, the one he wants to receive
    normally and the other obliviously. Alice sends the ciphertexts of both messages to Bob,
    encrypted with the corresponding public key. Bob decrypts the chosen ciphertext,
    receiving the message he wants.
    """

    def __init__(self, socket):
        # to send and receive messages
        self.socket = socket

    def receiver_side(self, choice):
        """
        Implement Bob's side of the oblivious transfer protocol with passive security.
        """

        # Bob wants message 0
        if choice == 0:
            pk0, sk0 = elgamal_generate_keys()
            pk1, _ = elgamal_generate_keys() # oblivious of secret key
            
        # Bob wants message 1
        else:
            pk0, _ = elgamal_generate_keys() # oblivious of secret key
            pk1, sk1 = elgamal_generate_keys()

        # send public keys to Alice
        self.socket.send((pk0, pk1))

        # receive the ciphertexts
        c0, c1 = self.socket.receive()

        # decrypt the chosen ciphertext
        if choice == 0:
            m = elgamal_decrypt(c0, sk0, pk0)
        else:
            m = elgamal_decrypt(c1, sk1, pk1)

        # close the socket
        self.socket.close()

        return m
    
    def sender_side(self, msg0, msg1):
        """
        Implement Alice's side of the oblivious transfer protocol with passive security.
        """

        # get the public keys from Bob
        h0, h1 = self.socket.receive()

        # encrypt the messages
        c0 = elgamal_encrypt(msg0, h0)
        c1 = elgamal_encrypt(msg1, h1)

        # send the ciphertexts to Bob
        self.socket.send((c0, c1))

        # close the socket
        self.socket.close()

In [237]:
# example of Oblivious Transfer with passive security

socket_bob = Socket(socket_type=zmq.REQ, address=f"tcp://{SERVER_HOST}:{SERVER_PORT}")
socket_alice = Socket(socket_type=zmq.REP, address=f"tcp://{SERVER_HOST}:{SERVER_PORT}")

def run_bob(socket_bob):
    # initialize Bob's side
    ot_bob = ObliviousTransfer_PS(socket_bob)

    # Bob wants message 1
    choice = 1

    # run Bob's side of the protocol
    received_message = ot_bob.receiver_side(choice)

    # show the received message
    print("Bob received:", received_message.decode("utf-8"))

    # close the socket
    socket_bob.close()

def run_alice(socket_alice):
    # initialize Alice's side
    ot_alice = ObliviousTransfer_PS(socket_alice)

    # get Alice's messages
    msg0 = b"Hello"
    msg1 = b"World"

    # run Alice's side of the protocol
    ot_alice.sender_side(msg0, msg1)

    # close the socket
    socket_alice.close()

# run Alice and Bob in separate threads
bob_thread = threading.Thread(target=run_bob, args=(socket_bob,))
alice_thread = threading.Thread(target=run_alice, args=(socket_alice,))

bob_thread.start()
alice_thread.start()

bob_thread.join()
alice_thread.join()

Bob received: World


## Active Security

Implementation of a 1-out-of-2 Oblivious Transfer protocol with active security.

In [238]:
class ObliviousTransfer_AS:
    """
    Implement the oblivious transfer protocol with active security.

    In active security, Alice sends two public keys to Bob, Bob will use the one linked to the message
    he wants to recieve to encrypt his key. Alice will decrypt it using both keys, obtaining two different
    keys. Alice will then encrypt both messages with the two keys and send them to Bob. Bob will be able
    to decrypt only the message linked to the key he chose.
    """

    def __init__(self, socket):
        # to send and receive messages
        self.socket = socket

    def sender_side(self, msg0, msg1):
        """
        Implement Alice's side of the oblivious transfer protocol with active security.
        """

        # generate two key pairs
        pk0, sk0 = elgamal_generate_keys()
        pk1, sk1 = elgamal_generate_keys()

        # send public keys to Bob
        self.socket.send((pk0, pk1))

        # recieve encrypted key from Bob
        c = self.socket.receive()

        # decrypt the key with both private keys
        k0 = elgamal_decrypt(c, sk0, pk0)[:32]
        k1 = elgamal_decrypt(c, sk1, pk1)[:32]

        # encrypt the messages with the keys
        c0 = aes_encrypt(msg0, k0)
        c1 = aes_encrypt(msg1, k1)

        # send the ciphertexts to Bob
        self.socket.send((c0, c1))

        # close the socket
        self.socket.close()

    def receiver_side(self, choice):
        """
        Implement Bob's side of the oblivious transfer protocol with active security.
        """

        # generate secret key
        k = aes_generate_key()

        # receive the public keys
        pk0, pk1 = self.socket.receive()

        # encrypt with the chosen public key
        if choice == 0:
            c = elgamal_encrypt(k, pk0)
        
        else:
            c = elgamal_encrypt(k, pk1)

        # send the encrypted key to Alice
        self.socket.send(c)

        # receive the ciphertexts
        c0, c1 = self.socket.receive()

        # decrypt the chosen ciphertext
        if choice == 0:
            m = aes_decrypt(c0, k)

        else:
            m = aes_decrypt(c1, k)

        # close the socket
        self.socket.close()

        return m

In [239]:
# example of Oblivious Transfer with active security

socket_bob = Socket(socket_type=zmq.REP, address=f"tcp://{SERVER_HOST}:{SERVER_PORT}")
socket_alice = Socket(socket_type=zmq.REQ, address=f"tcp://{SERVER_HOST}:{SERVER_PORT}")

def run_bob(socket_bob):
    # initialize Bob's side
    ot_bob = ObliviousTransfer_AS(socket_bob)

    # Bob wants message 1
    choice = 1

    # run Bob's side of the protocol
    received_message = ot_bob.receiver_side(choice)

    # show the received message
    print("Bob received:", received_message.decode("utf-8"))

    # close the socket
    socket_bob.close()

def run_alice(socket_alice):
    # initialize Alice's side
    ot_alice = ObliviousTransfer_AS(socket_alice)

    # get Alice's messages
    msg0 = b"Hello"
    msg1 = b"World"

    # run Alice's side of the protocol
    ot_alice.sender_side(msg0, msg1)

    # close the socket
    socket_alice.close()

# run Alice and Bob in separate threads
bob_thread = threading.Thread(target=run_bob, args=(socket_bob,))
alice_thread = threading.Thread(target=run_alice, args=(socket_alice,))

bob_thread.start()
alice_thread.start()

bob_thread.join()
alice_thread.join()

Bob received: World


# Circuit parsing

Circuits written in Bristol format.

Info about Bristol format here: https://nigelsmart.github.io/MPC-Circuits/old-circuits.html

Example circuits take from: https://tudatalib.ulb.tu-darmstadt.de/handle/tudatalib/3776?locale-attribute=en

In [240]:
def parse_circuit_file(filepath):

    with open(filepath, 'r') as file:
        lines = file.readlines()

    # first line contains amount of gates and of wires
    n_gates, n_wires = map(int, lines[0].split())

    # second line contains amount of wires of input1, input2 and output 
    n_input1, n_input2, n_output = map(int, lines[1].split())

    # process gates
    gates = []

    for line in lines[2:]:
        parts = line.split()

        if len(parts) == 0: # empty line
            continue

        if parts[0] == 1: # gate with 1 input
            # get components
            _, _, id_input, id_output, op = parts

            # converts ids to int
            id_input, id_output = map(int, (id_input, id_output))

            # append gate
            gates.append((op, (id_input, id_output)))

        else: # gate with 2 inputs
            # get components
            _, _, id_input1, id_input2, id_output, op = parts

            # converts ids to int
            id_input1, id_input2, id_output = map(int, (id_input1, id_input2, id_output))

            # append gate
            gates.append((op, (id_input1, id_input2, id_output)))

    parsed_circuit = {"n_gates": n_gates,
                      "n_wires": n_wires,
                      "n_input1": n_input1,
                      "n_input2": n_input2,
                      "n_output": n_output,
                      "gates": gates}

    return parsed_circuit

In [241]:
filepath = r"circuits\fullAdder.bristol"

circuit = parse_circuit_file(filepath)
circuit

{'n_gates': 5,
 'n_wires': 8,
 'n_input1': 2,
 'n_input2': 1,
 'n_output': 2,
 'gates': [('XOR', (0, 1, 3)),
  ('XOR', (2, 3, 7)),
  ('AND', (2, 3, 4)),
  ('AND', (0, 1, 5)),
  ('OR', (4, 5, 6))]}

# Garbled Circuits

In [242]:
class GarbledGate:
    def __init__(self, op, wire_keys):
        """
        Takes in input the operation, the wires and their keys and outputs
        a garbled table for the gate.
        """

        self.encrypt = verifiable_encrypt
        self.decrypt = verifiable_decrypt
        self.double_encrypt = double_verifiable_encrypt
        self.double_decrypt = double_verifiable_decrypt

        truth_table = self.generate_truth_table(op)
        key_table = self.generate_key_table(op, truth_table, wire_keys)

        self.garbled_table = self.generate_garbled_table(op, key_table)

    def generate_key_table(self, op, truth_table, wire_keys):
        """
        Take in input the truth table and the wire keeys and produce a table of encryption keys.
        """

        key_table = []

        if op == "INV":
            for entry in truth_table:
                key_table.append((wire_keys[0][entry[0]], wire_keys[1][entry[1]]))

        else:
            for entry in truth_table:
                key_table.append((wire_keys[0][entry[0]], wire_keys[1][entry[1]], wire_keys[2][entry[2]]))

        return key_table

    def generate_garbled_table(self, op, key_table):
        """
        Take in input the key table and return the garbled table.
        """

        garbled_table = []

        if op == "INV":
            for entry in key_table:
                garbled_table.append(self.encrypt(entry[1], entry[0]))

        else:
            for entry in key_table:
                garbled_table.append(self.double_encrypt(entry[2], entry[0], entry[1]))
        
        return garbled_table

    def generate_truth_table(self, op):
        if op == "AND":
            return [(0, 0, 0),
                    (0, 1, 0),
                    (1, 0, 0),
                    (1, 1, 1)]
        
        elif op == "XOR":
            return [(0, 0, 0),
                    (0, 1, 1),
                    (1, 0, 1),
                    (1, 1, 0)]

        elif op == "OR":
            return [(0, 0, 0),
                    (1, 0, 1),
                    (0, 1, 1),
                    (1, 1, 1)]

        elif op == "INV":
            return [(0, 1),
                    (1, 0)]
        
        else:
            print(f"Gate not supported: {op}")

In [243]:
class GarbledCircuit:
    def __init__(self, circuit):
        """
        Given a circuit initialize the Garbled circuit
        """

        # setup input data and functions
        self.circuit = circuit
        self.generate_key = get_random_bytes
        self.garbled_table_bytes = 16

        # generate all keys linked to the wires
        self.wire_keys = self.generate_wire_keys(self.circuit["n_wires"])

        # generate all garble gates
        self.garbled_gates = self.generate_garbled_gates(self.circuit["gates"])

    def generate_wire_keys(self, n_wires):
        """
        Creates for each wire in the circuit a tuple of two keys, one for bit 1 and
        the other for bit 0.
        """

        return [(self.generate_key(self.garbled_table_bytes), self.generate_key(self.garbled_table_bytes)) for _ in range(n_wires)]
    
    def generate_garbled_gates(self, gates):
        """
        Given the list of gates and their information creates a list of garbled gates.
        """

        # initialize garbled gates list
        garbled_gates = []

        # populate the list
        for gate in gates:
            # get specifics from gate
            op, wires = gate

            # get keys for the required wires
            wire_keys = [self.wire_keys[wire] for wire in wires]

            # create GarbledGate object
            garbled_gates.append(GarbledGate(op, wire_keys))
        
        return garbled_gates

In [251]:
######################################
# EXAMPLE OF GARBLED GATE DECRYPTION #
######################################

# take example circuit
filepath = r"circuits\fullAdder.bristol"

# parse circuit
circuit = parse_circuit_file(filepath)

# create garbled circuit
garbled_circuit = GarbledCircuit(circuit)

# take first gate (XOR) and show key for output = 0
print(garbled_circuit.wire_keys[3][0])

# take first garbled circuit and show key obtained with input (0,0)
print(double_verifiable_decrypt(garbled_circuit.garbled_gates[0].garbled_table[0], garbled_circuit.wire_keys[0][0], garbled_circuit.wire_keys[1][0]))

# try to ungarble the same entry of the table using wrong bit for first input
print(double_verifiable_decrypt(garbled_circuit.garbled_gates[0].garbled_table[0], garbled_circuit.wire_keys[0][1], garbled_circuit.wire_keys[1][0]))

b'\xfe\x1a\x8e\x10\xcf\xc1z\xbf\x14\xbe<\x8a_u\x8a\r'
b'\xfe\x1a\x8e\x10\xcf\xc1z\xbf\x14\xbe<\x8a_u\x8a\r'
None
